In [ ]:
!pip install accelerate -U
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch
import transformers
import pandas as pd
!pip install peft
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_int8_training, get_peft_model_state_dict, PeftModel
from sklearn.utils import shuffle
!pip install datasets
from datasets import Dataset
import transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

In [ ]:
df = pd.read_excel('/content/ready_data.xlsx')
df.rename(columns={'Input_text': 'input_text', '0utput_text': 'output_text'}, inplace=True)

df.head()

,input_text,output_text
0,item 10. directors and executive officers of ...,"Vaughn R. Blake, aged 43, Director (Service: M..."
1,item 10. trustees and executive officers of t...,"Victor H. Schlesinger, aged 68, Chairman and T..."
2,item 10. directors and executive officers of ...,"David J. Eskra, aged 52, Chairman of the Board..."
3,item 10. directors and executive officers of ...,"Robert J. Donachie, aged 65, Director (Service..."
4,item 10. directors and executive officers of ...,"Richard H. Koontz, aged 53, Chairman of the Bo..."


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import tqdm as notebook_tqdm
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_int8_training, get_peft_model_state_dict, PeftModel
from sklearn.utils import shuffle

token = "Your Token"

In [ ]:
context = "Your context goes here."
query = "Your query goes here."

In [ ]:
answers = llama_model.predict(query=query, documents=[context])
print(answers)

AttributeError: ignored

In [ ]:
!pip install accelerate==0.20.1



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.25.0
    Uninstalling accelerate-0.25.0:
      Successfully uninstalled accelerate-0.25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.6.2 requires accelerate>=0.21.0, but you have accelerate 0.20.1 which is incompatible.


In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import tqdm as notebook_tqdm
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_int8_training, get_peft_model_state_dict, PeftModel
from sklearn.utils import shuffle

def create_peft_config(model):
  peft_cofig = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  inference_mode=False,
  r=8,
  lora_alpha=16,
  lora_dropout=0.05,
  target_modules=['q_proj', 'v_proj'],
  )
  model = prepare_model_for_int8_training(model)
  model.enable_input_require_grads()
  model = get_peft_model(model, peft_cofig)
  model.print_trainable_parameters()
  return model, peft_cofig

#model_name = 'TinyPixel/Llama-2-7B-bf16-sharded'
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name,token=token)
model = AutoModelForCausalLM.from_pretrained(model_name,token=token)
#model.to(device)

model, lora_config = create_peft_config(model)

def generate_prompt(data_point):
  return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
  ### Instruction:
  Extract entity from the given input:
  ### Input:
  {data_point["input_text"]}
  ### Response:
  {data_point["output_text"]}"""

tokenizer.pad_token_id = 0
def tokenize(prompt, add_eos_token=True):
  result = tokenizer(
  prompt,
  truncation=True,
  max_length=128,
  padding=False,
  return_tensors=None,
  )
  if ( result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < 128 and add_eos_token
  ):
    result["input_ids"].append(tokenizer.eos_token_id)
    result["attention_mask"].append(1)
    result["labels"] = result["input_ids"].copy()
  return result



def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenize(full_prompt)
  return tokenized_full_prompt



tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
# !pip install accelerate==0.20.1
# training_arguments = transformers.TrainingArguments(
#  per_device_train_batch_size=1, gradient_accumulation_steps=16,
#  learning_rate=4e-05,
#  logging_steps=100,
#  optim="adamw_torch",
#  evaluation_strategy="steps",
#  save_strategy="steps",
#  eval_steps=100,
#  save_steps=100,
#  output_dir="saved_models/"
#  )
# total_ds = shuffle(df, random_state=42)
# total_train_ds = total_ds.head(90)
# total_test_ds = total_ds.tail(10)


# total_train_ds_hf = Dataset.from_pandas(total_train_ds)
# total_test_ds_hf = Dataset.from_pandas(total_test_ds)

# tokenized_tr_ds = total_train_ds_hf.map(generate_and_tokenize_prompt)
# tokenized_te_ds = total_test_ds_hf.map(generate_and_tokenize_prompt)
# data_collator = transformers.DataCollatorForSeq2Seq(tokenizer)
# trainer = transformers.Trainer(model=model, tokenizer=tokenizer,
#             train_dataset=tokenized_tr_ds, eval_dataset=tokenized_te_ds, args=training_arguments,
#                data_collator=data_collator)
# !pip install accelerate==0.20.1

import transformers
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq, Trainer
import torch
torch.backends.cuda.max_split_size_mb = 1024
torch.cuda.empty_cache()
# Define your training arguments
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=4e-05,
    logging_steps=100,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=50,
    save_steps=50,
    output_dir="saved_models/"
)

# Assuming 'df' is your DataFrame
total_ds = shuffle(df, random_state=42)
total_train_ds = total_ds.head(90)
total_test_ds = total_ds.tail(10)

# Convert data to Hugging Face datasets
total_train_ds_hf = Dataset.from_pandas(total_train_ds)
total_test_ds_hf = Dataset.from_pandas(total_test_ds)

# Tokenize the datasets
tokenized_tr_ds = total_train_ds_hf.map(generate_and_tokenize_prompt)
tokenized_te_ds = total_test_ds_hf.map(generate_and_tokenize_prompt)

data_collator = DataCollatorForSeq2Seq(tokenizer)





Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Initialize and train the Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_tr_ds,
    eval_dataset=tokenized_te_ds,
    args=training_arguments,
    data_collator=data_collator
)

trainer.train()


OutOfMemoryError: ignored

In [ ]:
with torch.autocast("cuda"):
       trainer.train()